# Model Examples for Radar Polarimetry Review

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import cm
%matplotlib inline

# Effective medium model imports
from effmed.lib.matrix_model import effective_medium
from effmed.lib.supplemental import dB, rotational_transform, \
                                    coherence, phase_gradient2d

### Example 1: Uniform Anisotropic Transmission (no rotation)

In [ ]:
# Initialize the model at a chosen center frequency
em = effective_medium()
em.system_setup(fc=300e6)

# Set the material properties
lams = [0.,0.1,0.9]
em.ice_properties(T=253.,epsr=3.15,lam=lams)

# Model domain (one beat length)
dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(lams[1]-lams[0])/(2.*np.sqrt(em.epsr)))
H = 2.*np.pi/dphidz
layer_dz = 1.
zs = np.arange(.1,H+8.*layer_dz,layer_dz)

# Solve model
em.solve(zs,H,0.,0.,lams)

# Rotate model result to all azimuths for a 2-d image
rotational_transform(em,n_thetas=401)
Θs,Ds = np.meshgrid(em.thetas,em.range)

# Co-polarized phase coherence and gradient from the model result
coherence(em)
phase_gradient2d(em)
dphi_dz = em.dphi_dz.copy()
dphi_dz[dphi_dz<-0.05] = -0.05
dphi_dz[dphi_dz>0.05] = 0.05

# Get ellipse variables to plot
E0x = abs(em.HH)
E0y = abs(em.HV)
ϕHH = np.angle(em.HH)
ϕHV = np.angle(em.HV)
ϕ = ϕHH-ϕHV
ϕ[ϕ<-np.pi] += 2.*np.pi
ϕ[ϕ>np.pi] -= 2.*np.pi
γ = np.arctan(E0y/E0x)

In [ ]:
plt.rcParams['axes.linewidth'] = 1.0

plt.figure(figsize=(10,6))
gs = GridSpec(4,8)

ax1 = plt.subplot(gs[:2,:2])
plt.tick_params(labelleft=True,labelbottom=False,bottom=False,labeltop=True,top=True)
plt.contourf(Θs,Ds,np.real(dB(em.HH**2.)),levels=np.linspace(-10,0,100),cmap='Greys_r',extend='min')
cbar = plt.colorbar(orientation='horizontal',location='bottom',label='Co-Polarized Power (dB)')
cbar.set_ticks(np.linspace(-10,0,3))
plt.xlabel('Azimuth ($\psi$)')
plt.ylabel('Depth')
ax1.xaxis.set_label_position('top') 
ax1.set_xticks([0,np.pi/2.,np.pi])
ax1.set_xticklabels(['0','π/2','π'])
plt.ylim(H,0)
ax1.set_yticks([min(zs),H/2,H])
ax1.set_yticklabels(['0','L/2','L'])
ax1.text(0.075,.8,'a',transform=ax1.transAxes,fontweight='bold',fontsize=14,
       bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=2.))

ax2 = plt.subplot(gs[:2,2:4])
plt.tick_params(labelleft=False,labelbottom=False,bottom=False,labeltop=True,top=True)
ax2.set_xticks([0,np.pi/2.,np.pi])
plt.contourf(Θs,Ds,np.real(dB(em.HV**2.)),levels=np.linspace(-10,0,100),cmap='Greys_r',extend='min')
cbar = plt.colorbar(orientation='horizontal',location='bottom',label='Cross-Polarized Power (dB)')
cbar.set_ticks(np.linspace(-10,0,3))
plt.xlabel('Azimuth ($\psi$)')
ax2.xaxis.set_label_position('top') 
ax2.set_xticks([0,np.pi/2.,np.pi])
ax2.set_xticklabels(['0','π/2','π'])
plt.ylim(H,0)
ax2.set_yticks([min(zs),H/2,H])
ax2.text(0.075,.8,'b',transform=ax2.transAxes,fontweight='bold',fontsize=14,
       bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=2.))

ax3 = plt.subplot(gs[:2,4:6])
plt.tick_params(labelleft=False,labelbottom=False,bottom=False,labeltop=True,top=True)
ax3.set_xticks([0,np.pi/2.,np.pi])
plt.contourf(Θs,Ds,np.angle(em.chhvv),cmap='twilight_shifted',levels=np.linspace(-np.pi,np.pi,100))
cbar = plt.colorbar(orientation='horizontal',location='bottom',label='Co-Polarized Phase ($ϕ_{HHVV}$)')
cbar.set_ticks(np.linspace(-np.pi,np.pi,3))
cbar.set_ticklabels(['-π','0','π'])
plt.xlabel('Azimuth ($\psi$)')
ax3.xaxis.set_label_position('top') 
ax3.set_xticks([0,np.pi/2.,np.pi])
ax3.set_xticklabels(['0','π/2','π'])
plt.ylim(H,0)
ax3.set_yticks([min(zs),H/2,H])
ax3.text(0.075,.8,'c',transform=ax3.transAxes,fontweight='bold',fontsize=14,
       bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=2.))

ax4 = plt.subplot(gs[:2,6:])
plt.tick_params(labelleft=False,labelbottom=False,bottom=False,labeltop=True,top=True)
ax4.set_xticks([0,np.pi/2.,np.pi])
plt.contourf(Θs,Ds,dphi_dz,cmap='RdYlBu_r',levels=np.linspace(-0.05,0.05,100))
cbar = plt.colorbar(orientation='horizontal',location='bottom',label='Phase Gradient ($\partial\phi/\partial z$)')
cbar.set_ticks([-0.05,0,0.05])
cbar.set_ticklabels(['-0.05','0','0.05'])
plt.xlabel('Azimuth ($\psi$)')
ax4.xaxis.set_label_position('top') 
ax4.set_xticks([0,np.pi/2.,np.pi])
ax4.set_xticklabels(['0','π/2','π'])
plt.ylim(H,0)
ax4.set_yticks([min(zs),H/2,H])
ax4.text(0.075,.8,'d',transform=ax4.transAxes,fontweight='bold',fontsize=14,
       bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=2.))

thidxs = [1,15,100,150]
didxs = [0,65,130,195,260,325,390,455,520]
labels=['g','h','i','j']
for i,thidx in enumerate(thidxs):
    ax5i = plt.subplot(gs[2:,i+4])
    ax5i.tick_params(labelleft=False,left=False,labelbottom=False,bottom=False)
    plt.ylim(-1,1)
    plt.xlim(-1,1)
    for j,didx in enumerate(didxs):
        lHV = abs(em.HV[didx,thidx])*np.sin(np.linspace(-np.pi,np.pi,50)-ϕHV[didx,thidx])
        lHH = abs(em.HH[didx,thidx])*np.sin(np.linspace(-np.pi,np.pi,50)-ϕHH[didx,thidx])
        plt.plot(lHH,lHV-.5*j,alpha=1.,c=cm.twilight_shifted((ϕ[didx,thidx]+np.pi)/(2*np.pi)))
    plt.axis('equal')
    ax5i.text(0.05,.95,labels[i],transform=ax5i.transAxes,fontweight='bold',fontsize=14,
       bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=2.))
    for spine in ax5i.spines:
        ax5i.spines[spine].set_visible(False)
    
ax6 = plt.subplot(gs[2:,:2])
plt.tick_params(labelleft=True,labelbottom=False,bottom=False,top=True)
ax6.set_xticks([0,np.pi/2.,np.pi])
plt.contourf(Θs,Ds,γ,cmap='Greys_r',levels=np.linspace(0,np.pi/2.,100))
cbar = plt.colorbar(orientation='horizontal',label='Ellipse Rotation ($\gamma$)')
cbar.set_ticks(np.linspace(0,np.pi/2.,3))
cbar.set_ticklabels(['0','π/4','π/2'])
#plt.ylim(max(dat.range),min(dat.range))
plt.ylabel('Depth')
plt.ylim(H,0)
ax6.set_yticks([min(zs),H/2,H])
ax6.set_yticklabels(['0','L/2','L'])
ax6.text(0.075,.8,'e',transform=ax6.transAxes,fontweight='bold',fontsize=14,
       bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=2.))

ax7 = plt.subplot(gs[2:,2:4])
plt.tick_params(labelleft=False,labelbottom=False,bottom=False,top=True)
ax7.set_xticks([0,np.pi/2.,np.pi])
plt.contourf(Θs,Ds,ϕ,cmap='twilight_shifted',levels=np.linspace(-np.pi,np.pi,100))
cbar = plt.colorbar(orientation='horizontal',label='Ellipse Phase Delay ($\phi_{ellipse}$)')
cbar.set_ticks(np.linspace(-np.pi,np.pi,3))
cbar.set_ticklabels(['-π','0','π'])
for i,th in enumerate(thidxs):
    plt.arrow(em.thetas[th],-60,0,30,color='k',clip_on=False,head_width=.05,head_length=20)
    if i == 0:
        plt.text(em.thetas[th]-.02,-70,labels[i],ha='center',size=10)
    elif i == 1:
        plt.text(em.thetas[th]+.02,-70,labels[i],ha='center',size=10)
    else:
        plt.text(em.thetas[th],-70,labels[i],ha='center',size=10)
plt.xlim(min(em.thetas),max(em.thetas))
plt.ylim(H,0)
ax7.set_yticks([min(zs),H/2,H])
ax7.text(0.075,.8,'f',transform=ax7.transAxes,fontweight='bold',fontsize=14,
       bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=2.))

plt.tight_layout()

## Example 2: Uniform Anisotropy with Rotation

In [ ]:
# Initialize the model at a chosen center frequency
em = effective_medium()
em.system_setup(fc=300e6)

# Set the material properties
lams = [.15,.35,0.5]
em.ice_properties(T=253.,epsr=3.15,lam=lams)

# Model domain (two beat lengths)
dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(lams[1]-lams[0])/(2.*np.sqrt(em.epsr)))
H = 4.*np.pi/dphidz
layer_dz = 2.
zs = np.arange(1,H,layer_dz)

# Define model layers (with a girdle rotation)
thetas = np.zeros(len(zs))
psis = .01+np.zeros_like(zs)
psis[zs>200] = .6
dzs = layer_dz*np.ones(len(zs))
lams = np.tile(lams,(len(zs),1))

# Solve model
em.solve(zs,dzs,thetas,psis,lams)

# Rotate model result to all azimuths for a 2-d image
rotational_transform(em,n_thetas=401)
Θs,Ds = np.meshgrid(em.thetas,em.range)

# Co-polarized phase coherence and gradient from the model result
coherence(em)

In [ ]:
fig = plt.figure(figsize=(4.5,3))

labels = ['a','b']
cred = 'darkmagenta'
cgreen = 'forestgreen'

ax1 = plt.subplot(1,2,1,facecolor='k')
im1 = plt.pcolormesh(Θs,Ds,np.real(dB(em.HV**2.)),cmap='Greys_r',vmin=-10,vmax=0.,rasterized=True)
cbar1 = plt.colorbar(label='Cross-Pol Power (dB)',extend='min',location='top',ticks=[-10,-5,0])
cbar1.ax.set_xticklabels(['-π','0','π'])
cbar1.ax.xaxis.tick_top()
cbar1.ax.xaxis.set_label_position('top') 
ax1.text(0.1,.85,labels[i*2],transform=ax1.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
       bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
#plt.plot(dat.cpe,Ds,'.',c=cred,ms=.5,rasterized=True)
#plt.plot(dat.cpe+np.pi/2.,Ds,'.',c=cred,ms=.5,rasterized=True)
#plt.plot(dat.cpe-np.pi/2.,Ds,'.',c=cred,ms=.5,rasterized=True)
plt.plot(psis,zs,'.',c=cgreen,ms=.5,rasterized=True)
plt.plot(psis+np.pi/2.,zs,'.',c=cgreen,ms=.5,rasterized=True)
plt.ylim(H,0)
plt.yticks([0,H/4,H/2,3*H/4,H])
plt.xticks([0,np.pi/2.,np.pi])
if i in [0]:
    ax1.set_ylabel('Depth')
    ax1.set_yticklabels(['0','','L','','2L'])
else:
    ax1.tick_params(labelleft=False)    

ax2 = plt.subplot(1,2,2)
im2 = plt.pcolormesh(Θs,Ds,np.angle(em.chhvv),cmap='twilight_shifted',vmin=-np.pi,vmax=np.pi,rasterized=True)
cbar2 = plt.colorbar(label='Phase ($\phi_{HHVV}$)',location='top',ticks=[-np.pi,0,np.pi])
cbar2.ax.set_xticklabels(['-π','0','π'])
cbar2.ax.xaxis.tick_top()
cbar2.ax.xaxis.set_label_position('top') 
ax2.text(0.1,.875,labels[i*2+1],transform=ax2.transAxes,fontweight='bold',fontsize=14,ha='center',va='center',
       bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
plt.ylim(H,0)
plt.yticks([0,H/4,H/2,3*H/4,H])
plt.xticks([0,np.pi/2.,np.pi])
plt.tick_params(labelleft=False)

ax1.set_xlim(0,np.pi)
ax1.set_xlabel('Azimuth (ψ)')
ax1.set_xticklabels(['0','π/2','π'])
ax2.set_xlabel('Azimuth (ψ)')
ax2.set_xticklabels(['0','π/2','π'])

plt.tight_layout()

### Example 3: Anisotropic Reflectivity

In [ ]:
# Initialize the model at a chosen center frequency
em = effective_medium()
em.system_setup(fc=300e6)

# Set the material properties
lams = np.array([.333,.334,.333])   # no girdle
gammas = np.array([1.,.3])          # anisotropic reflectivity
em.ice_properties(T=253.,epsr=3.15,lam=lams)

# Model domain (one beat length)
dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(.1)/(2.*np.sqrt(em.epsr)))
H = 2.*np.pi/dphidz
zs = np.arange(10,H)

# Solve model
em.solve(zs,H,0.,0.,lams,gammas=gammas)

# Rotate model result to all azimuths for a 2-d image
rotational_transform(em,n_thetas=401)
Θs,Ds = np.meshgrid(em.thetas,em.range)

# Co-polarized phase coherence and gradient from the model result
coherence(em)

# Rename model class so that we can create a second
em1 = em

In [ ]:
# Initialize the model at a chosen center frequency
em = effective_medium()
em.system_setup(fc=300e6)

# Set the material properties
lams = np.array([0.,.1,0.9])      # girdle
gammas = np.array([1.,.3])        # anisotropic reflectivity
em.ice_properties(T=253.,epsr=3.15,lam=lams)

# Model domain (one beat length)
dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(lams[1]-lams[0])/(2.*np.sqrt(em.epsr)))
H = 2.*np.pi/dphidz
zs = np.arange(10,H)

# Solve model
em.solve(zs,H,0.,0.,lams,gammas=gammas)

# Rotate model result to all azimuths for a 2-d image
rotational_transform(em,n_thetas=401)
Θs,Ds = np.meshgrid(em.thetas,em.range)

# Co-polarized phase coherence and gradient from the model result
coherence(em)

# Rename model class
em2 = em

In [ ]:
fig = plt.figure(figsize=(8,6))
gs = GridSpec(3,4)

ems = [em1,em2]
labels = ['a','b','c','d','e','f','g','h','i']

for i,em in enumerate(ems):
    Θs,Ds = np.meshgrid(em.thetas,em.range)

    ax1 = plt.subplot(gs[i,0],facecolor='k')
    im1 = plt.pcolormesh(Θs,Ds,np.real(dB(em.HH**2.)),cmap='Greys_r',vmin=-20,vmax=0.,rasterized=True)
    ax1.text(0.1,.85,labels[i*3],transform=ax1.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
           bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
    plt.ylim(H,10)
    plt.yticks([10,(H-10)/2.,H])
    plt.xticks([0,np.pi/2.,np.pi])
    if i != 1:
        ax1.set_xticklabels(['','',''])
        plt.tick_params(labelleft=False)
    else:
        ax1.set_xticklabels(['0','π/2','π'])
        ax1.set_yticklabels(['0','L/2','L'])
        ax1.set_xlabel('Azimuth ($\psi$)')
        ax1.set_ylabel('Depth')
        #ax3.yaxis.set_label_position('right') 
    
    ax2 = plt.subplot(gs[i,1],facecolor='k')
    plt.tick_params(labelleft=False)
    im2 = plt.pcolormesh(Θs,Ds,np.real(dB(em.HV**2.)),cmap='Greys_r',vmin=-20,vmax=0.,rasterized=True)
    ax2.text(0.1,.85,labels[i*3+1],transform=ax2.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
           bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
    plt.ylim(H,10)
    plt.yticks([10,(H-10)/2.,H])
    plt.xticks([0,np.pi/2.,np.pi])
    ax2.set_xticklabels(['','',''])

    ax3 = plt.subplot(gs[i,2])
    plt.tick_params(labelleft=False)
    im3 = plt.pcolormesh(Θs,Ds,np.angle(em.chhvv),cmap='twilight_shifted',vmin=-np.pi,vmax=np.pi,rasterized=True)
    ax3.text(0.1,.85,labels[i*3+2],transform=ax3.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
           bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
    plt.ylim(H,10)
    plt.yticks([10,(H-10)/2.,H])
    plt.xticks([0,np.pi/2.,np.pi])
    ax3.set_xticklabels(['','',''])

# -----------------------------------------------------------------------------------
# Colorbars
# -----------------------------------------------------------------------------------

fig.subplots_adjust(top=0.8)
cbar1_ax = fig.add_axes([.14, 0.825, .15, 0.02])
cbar2_ax = fig.add_axes([.34, 0.825, .15, 0.02])
cbar3_ax = fig.add_axes([.54, 0.825, .15, 0.02])
cbar1 = fig.colorbar(im1,cax=cbar1_ax,label='Co-Pol Power (dB)',extend='min',orientation='horizontal',ticks=[-20,-10,0])
cbar2 = fig.colorbar(im2,cax=cbar2_ax,label='Cross-Pol Power (dB)',extend='min',orientation='horizontal',ticks=[-20,-10,0])
cbar3 = fig.colorbar(im3,cax=cbar3_ax,label='Phase Angle ($\phi_{HHVV}$)',orientation='horizontal',ticks=[-np.pi,0,np.pi])
cbar3.ax.set_xticklabels(['-π','0','π'])

for cbar in [cbar1,cbar2,cbar3]:
    cbar.ax.xaxis.tick_top()
    cbar.ax.xaxis.set_label_position('top') 

plt.show()

In [ ]:
# Plot the relationship between the reflection ratio, r, and azimuthal distance between co-polarized nodes
# (Ershadi et al. 2023)

plt.figure(figsize=(4,4))
ax1 = plt.subplot(111)
ψs = np.linspace(0.00001,np.pi,1000)
rs = 10.*np.log10(1/(np.tan(ψs/2)**(2.)))
plt.plot(rs,ψs,'k')
plt.xlim(-40,40)
plt.ylim(0,np.pi)
plt.yticks([0,np.pi/2.,np.pi])
ax1.set_yticklabels(['0','π/2','π'])
plt.ylabel('Azimuthal Distance ($\psi_{AD}$)')
plt.xlabel('Reflection Ratio (dB)');